# 080420 Extracción en tiempo real de tweets por usuario VII

maqueta

In [4]:
#%load_ext watermark
%watermark -a "Christian Castro" -u -d -p numpy,pandas,matplotlib
%watermark -a "Propiedad de DataIntelligence" 

Christian Castro 
last updated: 2020-04-09 

numpy 1.18.2
pandas 1.0.1
matplotlib 3.1.3
Propiedad de DataIntelligence


In [ ]:
# Importamos las librerías necesarias:

from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import re
import time

# Declaramos nuestras Twitter API Keys:
access_token = '1230251564616515586-2KqPsCG2mIJp3irRjENgHpCfQUxTUg'
access_token_secret = '6PJfMtYGY7w6csiIX9m1S5jFEKNZ3hE9PVkHKeN1S14iM'
consumer_key = 'koO4XqTuWFr5ADGcE8kjIkVoU'
consumer_secret = '3F4sk9qU8zbKBROuLPUUj1uvE2YuhseXPe0ahMQoivg4icN5bL'

#auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
#auth.set_access_token(access_token, access_token_secret)
#api = tweepy.API(auth)

# Se debe utilizar apscheduler
# https://medium.com/better-programming/introduction-to-apscheduler-86337f3bb4a6
'''Advanced Python Scheduler (APScheduler) es una biblioteca de Python que te permite programar 
tu código de Python para que se ejecute más tarde, ya sea solo una vez o periódicamente. 
Puede agregar nuevos trabajos o eliminar los antiguos sobre la marcha, según lo desees. 
Si almacena sus trabajos en una base de datos, también sobrevivirán a los reinicios del planificador 
y mantendrán su estado. Cuando se reinicia el planificador, ejecutará todos los trabajos que debería 
haber ejecutado mientras estaba desconectado'''

#from apscheduler.schedulers.background import BackgroundScheduler

# Inicializamos la variable global: 
tweet_count = 0

# Ingresamos el número de tweets a ser descargados:
n_tweets = 10

# Crea la clase que manejará el stream de tweets:
class StdOutListener(StreamListener):
    
    #sched = BackgroundScheduler(daemon=True)
    #sched.add_job(lambda : sched.on_data(),'interval',seconds=500)
    #sched.start()
    #sched.shutdown()
     
    def on_data(self, data):
        global tweet_count
        global n_tweets
        global stream
        if tweet_count < n_tweets:
            
            # Imprimimos en pantalla los tweets descargados:
            print(data)
            tweet_count += 1
            
            # Guardar el archivo data como txt:
            # el parámetro 'a' permite ir escribiendo debajo de lo ya existente en el txt:
            # https://www.geeksforgeeks.org/reading-writing-text-files-python/
            write_on_txt=open("twitter_data.txt",'a')
            write_on_txt.write(data)
            write_on_txt.close()  

            return True
        
        else:
            stream.disconnect()
      
    def on_error(self, status):
        print(status)

# Manejamos la autenticación de Twitter y la conexión a la API de Streaming de Twitter:
l = StdOutListener()
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
stream = Stream(auth, l)

# Integramos un flitro para que busque un usuario, en éste caso de la WHO:
# id del usuario: WHO: 14499829
usuario = ['14499829']
stream.filter(follow=usuario)

{"created_at":"Fri Apr 10 17:51:46 +0000 2020","id":1248670017559896065,"id_str":"1248670017559896065","text":"RT @WHO: \"We are particularly concerned by the large numbers of infections reported among #healthworkers. \n\nIn some countries there are rep\u2026","source":"\u003ca href=\"http:\/\/twitter.com\/download\/android\" rel=\"nofollow\"\u003eTwitter for Android\u003c\/a\u003e","truncated":false,"in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"in_reply_to_user_id_str":null,"in_reply_to_screen_name":null,"user":{"id":1022516819628834816,"id_str":"1022516819628834816","name":"Pol Alcal\u00e0 \ud83d\udc36","screen_name":"Pol_alcala3","location":"Matar\u00f3, Espa\u00f1a","url":null,"description":"Voluntari de la Societat Protectora d'Animals de Matar\u00f3\n#spAm #adoptanocompris Voluntari de les Santes #cultura","translator_type":"none","protected":false,"verified":false,"followers_count":35,"friends_count":93,"listed_count":1,"favourites_cou

In [9]:
import json
import pandas as pd
tweets_data_path = "twitter_data.txt"  
tweets_data = []  
tweets_file = open(tweets_data_path, "r")  
for line in tweets_file:  
    try:  
        tweet = json.loads(line)  
        tweets_data.append(tweet)  
    except:  
        continue
        
tweets = pd.DataFrame()
tweets['Texto del tweet'] = list(map(lambda tweet: 
tweet['text'], tweets_data))
tweets['Usuario'] = list(map(lambda tweet:
tweet['user']['screen_name'], tweets_data))
tweets['Fecha'] = list(map(lambda tweet:
tweet['created_at'], tweets_data))

tweets

,Texto del tweet,Usuario,Fecha
0,@WHO Propaganda to make @WHO look like they di...,LyndaMarie8888,Fri Apr 10 01:37:18 +0000 2020
1,@WHO this is pretty accurate..,Hezander1,Fri Apr 10 01:37:20 +0000 2020
2,@WHO SUCKS &amp; is guilty of BS for the #Chin...,labachus,Fri Apr 10 01:37:28 +0000 2020
3,RT @WHO: You can catch #COVID19 no matter how ...,MrBazingga,Fri Apr 10 01:37:33 +0000 2020
4,@WHO Biggest event: WHO lied to the world that...,CrystalCar2,Fri Apr 10 01:38:01 +0000 2020
...,...,...,...
99,"RT @WHO: ""Today I convened a meeting of the Em...",NiliMajumder,Fri Apr 10 17:06:46 +0000 2020
100,"RT @WHO: ""I know that some countries are alrea...",claujisan,Fri Apr 10 17:06:48 +0000 2020
101,@WHO @DrTedros CCP is NOT in this together! Th...,SkindogC,Fri Apr 10 17:06:51 +0000 2020
102,"RT @WHO: ""Today I convened a meeting of the Em...",mariasstories,Fri Apr 10 17:07:09 +0000 2020


In [10]:
reversed_tweets = tweets.iloc[::-1]
reversed_tweets

,Texto del tweet,Usuario,Fecha
103,RT @WHO: Media briefing on #COVID19 with @DrTe...,ShineOfColors,Fri Apr 10 17:07:25 +0000 2020
102,"RT @WHO: ""Today I convened a meeting of the Em...",mariasstories,Fri Apr 10 17:07:09 +0000 2020
101,@WHO @DrTedros CCP is NOT in this together! Th...,SkindogC,Fri Apr 10 17:06:51 +0000 2020
100,"RT @WHO: ""I know that some countries are alrea...",claujisan,Fri Apr 10 17:06:48 +0000 2020
99,"RT @WHO: ""Today I convened a meeting of the Em...",NiliMajumder,Fri Apr 10 17:06:46 +0000 2020
...,...,...,...
4,@WHO Biggest event: WHO lied to the world that...,CrystalCar2,Fri Apr 10 01:38:01 +0000 2020
3,RT @WHO: You can catch #COVID19 no matter how ...,MrBazingga,Fri Apr 10 01:37:33 +0000 2020
2,@WHO SUCKS &amp; is guilty of BS for the #Chin...,labachus,Fri Apr 10 01:37:28 +0000 2020
1,@WHO this is pretty accurate..,Hezander1,Fri Apr 10 01:37:20 +0000 2020


In [11]:
import csv
#reversed_tweets.to_excel('reversed_tweets.csv', sheet_name='primera')
tweets.to_csv(r'reversed_tweets.csv', index = False)

#### Ahora se debe borrar el contenido del txt: